In [ ]:
import boto3

ec2 = boto3.client('ec2')
iam = boto3.client('iam')

In [ ]:
assume_role_policy_doc = """{
    "Version": "2012-10-17", 
    "Statement": [
        {
            "Action": "sts:AssumeRole", 
            "Effect": "Allow", 
            "Principal": {
                "Service": "ec2.amazonaws.com"
            }
        }
    ]
}"""

In [ ]:
response = iam.create_role(
    RoleName='S3-Super-User',
    AssumeRolePolicyDocument=assume_role_policy_doc
)

print response

In [ ]:
attach_policy_response = iam.attach_role_policy(
    RoleName='S3-Super-User',
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess'
)

print attach_policy_response

In [ ]:
# Now we need to create an instance profile, to which we'll add the role created
# above. The ec2 instance will be launched with a reference to the profile
instance_profile_response = iam.create_instance_profile(
    InstanceProfileName='EC2-S3-Profile'
)

print instance_profile_response

In [ ]:
# Add our role to the profile
add_role_resp = iam.add_role_to_instance_profile(
    InstanceProfileName='EC2-S3-Profile',
    RoleName='S3-Super-User'
)

print add_role_resp

In [ ]:
my_session = boto3.session.Session()
my_region = my_session.region_name
print my_region

In [ ]:
# AMIs are unique per region. Here we will look up our AMI based on our region
amis = {}
amis['us-east-1'] = 'ami-0b33d91d'
amis['us-east-2'] = 'ami-446f3521'
amis['us-west-1'] = 'ami-9fadf8ff'
amis['us-west-2'] = 'ami-7abc111a'
amis['eu-west-1'] = 'ami-a1491ad2'
amis['ca-central-1'] = 'ami-ebed508f'

ami_id = amis[my_region]
print ami_id

In [ ]:
#Create security group
response = ec2.create_security_group(
    GroupName='ssh_sg',
    Description='Use for ssh to public server'
)

sshSgID = response['GroupId']
print 'created security group ', sshSgID

# Allow ingress for ssh
response = ec2.authorize_security_group_ingress(
    GroupId=sshSgID,
    IpProtocol='tcp',
    FromPort=22,
    ToPort=22,
    CidrIp='0.0.0.0/0'
)

print response

In [ ]:
import os

keyName = os.environ["KEYNAME"]
print keyName

In [ ]:
# Launch instance - default VPC, default subnet, etc.
launch_response = ec2.run_instances(
    KeyName = keyName,
    DryRun=False,
    ImageId=ami_id,
    MinCount=1,
    MaxCount=1,
    InstanceType='t2.micro',
    SecurityGroups=[
        'ssh_sg'
    ],
    IamInstanceProfile={
        'Name':'EC2-S3-Profile'
    }
)


instanceId = launch_response['Instances'][0]['InstanceId']

In [ ]:
# Wait for instance
print 'Waiting for launch of ', instanceId
waiter = ec2.get_waiter('instance_running')
waiter.wait(
    InstanceIds=[
        instanceId,
    ]
)
print 'Launch complete'

In [ ]:
# Get the dns name
response = ec2.describe_instances(
InstanceIds=[
        instanceId,
    ]
)

print response['Reservations'][0]['Instances'][0]['PublicDnsName']

In [ ]:
# Terminate the EC2 Instance
response = ec2.terminate_instances(
    InstanceIds=[
        instanceId,
    ]
)

print response

In [ ]:
print 'wait for it...'
waiter = ec2.get_waiter('instance_terminated')
waiter.wait(
    InstanceIds=[
        instanceId,
    ]
)
print 'proceed'

In [ ]:
response = iam.remove_role_from_instance_profile(
    InstanceProfileName='EC2-S3-Profile',
    RoleName='S3-Super-User'
)

print response

In [ ]:
response = iam.delete_instance_profile(
    InstanceProfileName='EC2-S3-Profile'
)

print response

In [ ]:
detach_response = iam.detach_role_policy(
    RoleName='S3-Super-User',
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess'
)

print detach_response

In [ ]:
del_response = iam.delete_role(
    RoleName='S3-Super-User'
)
print del_response

In [ ]:
# Clean up security group
response = ec2.delete_security_group(
    GroupId=sshSgID
)

print response